In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import CountEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold, TimeSeriesSplit
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
import catboost as cb
from catboost import CatBoostClassifier
import xgboost as xg
from xgboost import XGBClassifier

## 1. Download data. Implement train/validation/test split function.

In [2]:
df = pd.read_csv('training.csv')
df.head(10)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020
5,6,0,12/7/2009,ADESA,2004,5,MITSUBISHI,GALANT 4C,ES,4D SEDAN ES,...,8149.0,9451.0,NaN,NaN,19638,33619,FL,5600.0,0,594
6,7,0,12/7/2009,ADESA,2004,5,KIA,SPECTRA,EX,4D SEDAN EX,...,6230.0,8603.0,NaN,NaN,19638,33619,FL,4200.0,0,533
7,8,0,12/7/2009,ADESA,2005,4,FORD,TAURUS,SE,4D SEDAN SE,...,6942.0,8242.0,NaN,NaN,19638,33619,FL,4500.0,0,825
8,9,0,12/7/2009,ADESA,2007,2,KIA,SPECTRA,EX,4D SEDAN EX,...,9637.0,10778.0,NaN,NaN,21973,33619,FL,5600.0,0,482
9,10,0,12/7/2009,ADESA,2007,2,FORD,FIVE HUNDRED,SEL,4D SEDAN SEL,...,12580.0,14845.0,NaN,NaN,21973,33619,FL,7700.0,0,1633


In [3]:
df = df.sort_values(by='PurchDate', ascending=True).reset_index(drop=True)

In [4]:
def my_split_by_date_val(validation_date, test_date, data, date_col, inclusive=False):
    data[date_col] = pd.to_datetime(data[date_col])
    validation_date = pd.to_datetime(validation_date)
    test_date = pd.to_datetime(test_date)
    if validation_date >= test_date:
        raise ValueError("validation_date should be less than test_date")
    if inclusive:
        train = data[data[date_col] < validation_date]
        validation = data[(data[date_col] >= validation_date) & (data[date_col] < test_date)]
        test = data[data[date_col] >= test_date]
    else:
        train = data[data[date_col] < validation_date]
        validation = data[(data[date_col] >= validation_date) & (data[date_col] < test_date)]
        test = data[data[date_col] >= test_date] 
    return train, validation, test

In [5]:
validation_date=pd.to_datetime(df['PurchDate']).quantile(1/3)
test_date=pd.to_datetime(df['PurchDate']).quantile(2/3)
train_df, valid_df, test_df = my_split_by_date_val(validation_date,test_date,df,date_col='PurchDate',inclusive=True)
train_df['PurchDate'].max()

Timestamp('2009-09-14 00:00:00')

In [6]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols

['Auction',
 'Make',
 'Model',
 'Trim',
 'SubModel',
 'Color',
 'Transmission',
 'WheelType',
 'Nationality',
 'Size',
 'TopThreeAmericanName',
 'PRIMEUNIT',
 'AUCGUART',
 'VNST']

In [7]:
encoder = CountEncoder()
encoder.fit(train_df[categorical_cols])

,verbose,0
,cols,"['Auction', 'Make', ...]"
,drop_invariant,False
,return_df,True
,handle_unknown,'value'
,handle_missing,'value'
,min_group_size,None
,combine_min_nan_groups,True
,min_group_name,None
,normalize,False


In [8]:
train_df=train_df.copy()
valid_df=valid_df.copy()
test_df=test_df.copy()
train_df[categorical_cols]=encoder.transform(train_df[categorical_cols])
valid_df[categorical_cols]=encoder.transform(valid_df[categorical_cols])
test_df[categorical_cols]=encoder.transform(test_df[categorical_cols])
train_df

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
38,8899,0,2009-01-12,14038,2004,5,4766,166,424,160,...,7821.0,9413.0,24230,24230,18880,27542,2444,6900.0,0,1763
39,19212,0,2009-01-12,5224,2005,4,4250,308,3421,96,...,4577.0,5777.0,24230,24230,21053,95673,2547,4590.0,0,1506
40,8886,0,2009-01-12,14038,2005,4,284,53,232,5,...,6870.0,8754.0,24230,24230,18880,27542,2444,6100.0,0,1155
41,8887,0,2009-01-12,14038,2005,4,4766,328,3421,511,...,4031.0,4996.0,24230,24230,16044,27542,2444,3300.0,0,1918
42,8888,0,2009-01-12,14038,2003,6,319,19,2979,54,...,5615.0,6318.0,24230,24230,18880,27542,2444,5800.0,0,805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72819,27480,1,2009-09-09,5224,2003,6,4250,128,789,76,...,3444.0,4560.0,24230,24230,835,85009,2449,5385.0,0,2322
72820,46569,0,2009-09-09,14038,2006,3,2760,123,711,285,...,5496.0,6489.0,24230,24230,20928,33809,3387,4910.0,0,1272
72821,46570,0,2009-09-09,14038,2005,4,1437,682,267,166,...,7222.0,8777.0,24230,24230,20928,33809,3387,7610.0,0,1974
72822,46571,0,2009-09-09,14038,2006,3,5644,665,2979,1255,...,7683.0,8968.0,24230,24230,20928,33809,3387,8045.0,0,2152


In [9]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 24232 entries, 38 to 72823
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   RefId                              24232 non-null  int64         
 1   IsBadBuy                           24232 non-null  int64         
 2   PurchDate                          24232 non-null  datetime64[ns]
 3   Auction                            24232 non-null  int64         
 4   VehYear                            24232 non-null  int64         
 5   VehicleAge                         24232 non-null  int64         
 6   Make                               24232 non-null  int64         
 7   Model                              24232 non-null  int64         
 8   Trim                               24232 non-null  int64         
 9   SubModel                           24232 non-null  int64         
 10  Color                              242

In [10]:
X_train = train_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_train = train_df['IsBadBuy'].copy()
X_valid = valid_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_valid = valid_df['IsBadBuy'].copy()
X_train=X_train.fillna(X_train.mean())
X_valid=X_valid.fillna(X_train.mean())
X_test=test_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_test = test_df['IsBadBuy'].copy()
feature_columns = X_train.columns.tolist()

## 2. Implement Decision Tree Classifier and Decision Tree Regressor (MSE loss).

In [11]:
def find_best_split_clsn(self, X, y):
    best_gini = float('inf')
    best_feature = None
    best_threshold = None
    n_samples, n_features = X.shape
    for feature_idx in range(n_features):
        feature_values = np.unique(X[:, feature_idx])
        for threshold in feature_values:
            left_mask = X[:, feature_idx] <= threshold
            right_mask = X[:, feature_idx] > threshold
            left_y = y[left_mask]
            right_y = y[right_mask]
            if len(left_y) == 0 or len(right_y) == 0:
                continue
            left_gini = self._gini(left_y)
            right_gini = self._gini(right_y)
            n_left = len(left_y)
            n_right = len(right_y)
            total = n_left + n_right            
            weighted_gini = (n_left / total) * left_gini + (n_right / total) * right_gini
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature_idx
                best_threshold = threshold
    return best_feature, best_threshold, best_gini

In [12]:
def find_best_split_rgsn(self, X, y):
    best_mse = float('inf')
    best_feature = None
    best_threshold = None
    n_samples, n_features = X.shape
    for feature_idx in range(n_features):
        feature_values = np.unique(X[:, feature_idx])
        for threshold in feature_values:
            left_mask = X[:, feature_idx] <= threshold
            right_mask = X[:, feature_idx] > threshold
            left_y = y[left_mask]
            right_y = y[right_mask]
            if len(left_y) == 0 or len(right_y) == 0:
                continue
            left_mse = np.var(left_y)
            right_mse = np.var(right_y)
            n_left = len(left_y)
            n_right = len(right_y)
            total = n_left + n_right
            weighted_mse = (n_left / total) * left_mse + (n_right / total) * right_mse
            if weighted_mse < best_mse:
                best_mse = weighted_mse
                best_feature = feature_idx
                best_threshold = threshold
    return best_feature, best_threshold, best_mse

In [13]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None, probs=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.probs = probs
    
    def is_leaf(self):
        return self.value is not None

class my_DecisionTreeClassifier:
    def __init__(self, max_depth=100, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        self.n_classes_ = None
        
    def _gini(self, y):
        if len(y) == 0:
            return 0
        counts = np.bincount(y)
        probs = counts / len(y)
        return 1 - np.sum(probs ** 2)
    
    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def _calculate_probs(self, y):
        if len(y) == 0:
            return np.zeros(self.n_classes_) if self.n_classes_ else None
        counts = np.bincount(y, minlength=self.n_classes_)
        probs = counts / len(y)
        return probs
    
    def _find_best_split_clsn(self, X, y):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None
        n_samples, n_features = X.shape
        for feature_idx in range(n_features):
            feature_values = np.unique(X[:, feature_idx])
            for threshold in feature_values:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = X[:, feature_idx] > threshold
                left_y = y[left_mask]
                right_y = y[right_mask]
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                left_gini = self._gini(left_y)
                right_gini = self._gini(right_y)
                n_left = len(left_y)
                n_right = len(right_y)
                total = n_left + n_right            
                weighted_gini = (n_left / total) * left_gini + (n_right / total) * right_gini
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature_idx
                    best_threshold = threshold
        return best_feature, best_threshold, best_gini
    
    def _build_tree(self, X, y, depth=0):
        n_samples = len(y)
        node_probs = self._calculate_probs(y)
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split or 
            len(np.unique(y)) == 1):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, probs=node_probs)
        best_feature, best_threshold, best_gini = self._find_best_split_clsn(X, y)
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, probs=node_probs)
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = X[:, best_feature] > best_threshold
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        return Node(
            feature=best_feature, 
            threshold=best_threshold,
            left=left_subtree, 
            right=right_subtree,
            probs=node_probs
        )
    
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.n_classes_ = len(np.unique(y))
        self.root = self._build_tree(X, y)
        return self
    
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    def _traverse_tree_proba(self, x, node):
        if node.is_leaf():
            return node.probs
        if x[node.feature] <= node.threshold:
            return self._traverse_tree_proba(x, node.left)
        return self._traverse_tree_proba(x, node.right)
    
    def predict(self, X):
        X = np.array(X)
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def predict_proba(self, X):
        X = np.array(X)
        proba = np.array([self._traverse_tree_proba(x, self.root) for x in X])
        return proba

In [14]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None, probs = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.probs = probs

    def is_leaf(self):
        return self.value is not None
    
class my_DecisionTreeRegressor:
    def __init__(self, max_depth=100, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    
    def _build_tree(self, X, y, depth=0):
        n_samples = len(y)
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split):
            leaf_value = np.mean(y)
            return Node(value=leaf_value)
        best_feature, best_threshold, best_mse = self._find_best_split_rgsn(X, y)
        if best_feature is None:
            leaf_value = np.mean(y)
            return Node(value=leaf_value)
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = X[:, best_feature] > best_threshold
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        return Node(
            feature=best_feature, 
            threshold=best_threshold,
            left=left_subtree, 
            right=right_subtree
        )

    def _find_best_split_rgsn(self, X, y):
        best_mse = float('inf')
        best_feature = None
        best_threshold = None
        n_samples, n_features = X.shape
        for feature_idx in range(n_features):
            feature_values = np.unique(X[:, feature_idx])
            for threshold in feature_values:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = X[:, feature_idx] > threshold
                left_y = y[left_mask]
                right_y = y[right_mask]
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                left_mse = np.var(left_y)
                right_mse = np.var(right_y)
                n_left = len(left_y)
                n_right = len(right_y)
                total = n_left + n_right
                weighted_mse = (n_left / total) * left_mse + (n_right / total) * right_mse
                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_feature = feature_idx
                    best_threshold = threshold
        return best_feature, best_threshold, best_mse

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.root = self._build_tree(X, y)
        return self
    
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    def predict(self, X):
        X = np.array(X)
        return np.array([self._traverse_tree(x, self.root) for x in X])

In [15]:
model = my_DecisionTreeClassifier(max_depth=7, min_samples_split=2)
model.fit(X_train, y_train)
y_proba=model.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_valid, y_proba)
gini = 2 * auc - 1
gini


0.4329411995058723

In [16]:
model=DecisionTreeClassifier(max_depth=7, min_samples_split=2)
model.fit(X_train, y_train)
y_proba=model.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_valid, y_proba)
gini = 2 * auc - 1
gini

0.43417726062867223

### Таким образом, на данной выборке моя реализация DesicionTreeClassifier работает также, как реализованный в sklearn метод с точностью до сотых. 
##### Результат объясняется тем, что в моем алгоритме используется упрощенный подход к выбору порогов разбиения, при котором рассматриваются только уникальные значения признаков, в отличие от более сложной mid-point стратегии sklearn, вычисляющей середины между соседними значениями. Это может приводить к искусственным границам, не существующим в реальных данных.
##### Небольшое преимущество в коэффициенте gini также могло возникнуть из-за разной обработки листовых узлов и расчета вероятностей классов (sklearn использует сглаживание и взвешенное голосование).

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None, probs=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.probs = probs
    
    def is_leaf(self):
        return self.value is not None

class my_DecisionTreeClassifierRand:
    def __init__(self, max_depth=100, min_samples_split=2, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        self.n_classes_ = None
        self.rng = np.random.RandomState(random_state)
        
    def _gini(self, y):
        if len(y) == 0:
            return 0
        counts = np.bincount(y)
        probs = counts / len(y)
        return 1 - np.sum(probs ** 2)
    
    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def _calculate_class_probs(self, y):
        if len(y) == 0:
            return np.zeros(self.n_classes_) if self.n_classes_ else None
        counts = np.bincount(y, minlength=self.n_classes_)
        probs = counts / len(y)
        return probs

    def _find_best_split_extra_random(self, X, y):
        n_samples, n_features = X.shape
        n_random_features = max(1, int(np.sqrt(n_features)))
        random_features = self.rng.choice(n_features, size=n_random_features, replace=False)
        best_gini = float('inf')
        best_feature = None
        best_threshold = None
        for feature_idx in random_features:
            feature_values = X[:, feature_idx]
            unique_values = np.unique(feature_values)
            if len(unique_values) <= 10:
                candidate_thresholds = (unique_values[:-1] + unique_values[1:]) / 2
            else:
                n_candidates = min(50, len(unique_values) // 2)
                candidate_thresholds = self.rng.choice(unique_values, size=n_candidates, replace=False)
            for threshold in candidate_thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = X[:, feature_idx] > threshold
                if np.sum(left_mask) < 2 or np.sum(right_mask) < 2:
                    continue
                left_y = y[left_mask]
                right_y = y[right_mask]
                if len(left_y) == 0 or len(right_y) == 0:
                    continue       
                left_gini = self._gini(left_y)
                right_gini = self._gini(right_y)
                n_left = len(left_y)
                n_right = len(right_y)
                total = n_left + n_right            
                weighted_gini = (n_left / total) * left_gini + (n_right / total) * right_gini
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature_idx
                    best_threshold = threshold             
        return best_feature, best_threshold, best_gini
    
    def _build_tree(self, X, y, depth=0):
        n_samples = len(y)
        node_probs = self._calculate_class_probs(y)
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split or 
            len(np.unique(y)) == 1):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, probs=node_probs)
        best_feature, best_threshold, best_gini = self._find_best_split_extra_random(X, y)
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, probs=node_probs)
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = X[:, best_feature] > best_threshold
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        return Node(
            feature=best_feature, 
            threshold=best_threshold,
            left=left_subtree, 
            right=right_subtree,
            probs=node_probs
        )
    
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.n_classes_ = len(np.unique(y))
        self.root = self._build_tree(X, y)
        return self
    
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    def _traverse_tree_proba(self, x, node):
        if node.is_leaf():
            return node.probs
        if x[node.feature] <= node.threshold:
            return self._traverse_tree_proba(x, node.left)
        return self._traverse_tree_proba(x, node.right)
    
    def predict(self, X):
        X = np.array(X)
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def predict_proba(self, X):
        X = np.array(X)
        proba = np.array([self._traverse_tree_proba(x, self.root) for x in X])
        return proba

## 5. Implement the RandomForestClassifier

In [18]:
class my_RandomForestClassifier:
    def __init__(self, number_of_trees=100, max_depth=10, max_features=100, min_samples_split=2, random_state=None):
        self.number_of_trees = number_of_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_samples_split = min_samples_split  
        self.trees = []
        self.feature_indices_list = []
        self.random_state=random_state
        if self.random_state is not None:
            np.random.seed(self.random_state)
    
    def fit(self, X, y):
        self.trees = []
        self.feature_indices_list= []
        if isinstance(X, pd.DataFrame):
            X_values = X.values
            n_features = X.shape[1]
        else:
            X_values = np.array(X)
            n_features = X_values.shape[1]   
        y_values = np.array(y)
        n_samples = X_values.shape[0]
        max_features = min(self.max_features, n_features)
        for i in range(self.number_of_trees):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_bootstrap = X_values[indices]
            y_bootstrap = y_values[indices]
            feature_indices = np.random.choice(n_features, max_features, replace=False)
            X_bootstrap_subset = X_bootstrap[:, feature_indices]
            tree = my_DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split  
            )
            tree.fit(X_bootstrap_subset, y_bootstrap)
            self.trees.append(tree)
            self.feature_indices_list.append(feature_indices)
        return self
    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X_values = X.values
        else:
            X_values = np.array(X)   
        all_proba = []
        for tree, feature_indices in zip(self.trees, self.feature_indices_list):
            X_subset = X_values[:, feature_indices]
            tree_proba = tree.predict_proba(X_subset)
            all_proba.append(tree_proba)
        return np.mean(all_proba, axis=0)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [19]:
rf = my_RandomForestClassifier(
    number_of_trees=10,
    max_depth=7,
    random_state=21,
    max_features=100,
    min_samples_split=2
)
rf.fit(X_train, y_train)
rf_proba = rf.predict_proba(X_valid)[:,1]
auc = roc_auc_score(y_valid, rf_proba)
gini = 2 * auc - 1
gini

0.4666587160034983

In [20]:
rf=RandomForestClassifier( n_estimators=10,
    max_depth=7,
    max_features=100, 
    min_samples_split=2,
    random_state=21)
rf.fit(X_train, y_train)
rf_proba = rf.predict_proba(X_valid)[:,1]
auc = roc_auc_score(y_valid, rf_proba)
gini = 2 * auc - 1
gini

0.4688606492013141

## 6. Implement the GBDT classifier

In [21]:
class my_GBDTClassifier:
    def __init__(self, number_of_trees=100, learning_rate=0.1, max_depth=3, max_features=None, 
                 min_samples_split=2, random_state=21):
        self.number_of_trees = number_of_trees
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_samples_split = min_samples_split
        self.trees = []
        self.init_predict = None
        self.random_state=random_state
        if random_state is not None:
            np.random.seed(random_state)
    
    def _sigmoid(self, x):
        x = np.clip(x, -10, 10)
        return 1 / (1 + np.exp(-x))
    
    def _binary_cross_entropy(self, y_true, y_pred_logits):
        probs = self._sigmoid(y_pred_logits)
        gradients = probs - y_true
        return gradients
    
    def fit(self, X, y):
        if self.random_state is not None:
            np.random.seed(self.random_state)
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        n_samples, n_features = X.shape
        n_pos = np.sum(y == 1)
        n_neg = len(y) - n_pos
        self.init_predict = np.log(n_pos / n_neg)
        cur_predict = np.full(len(y), self.init_predict)
        for i in range(self.number_of_trees):
            gradients = self._binary_cross_entropy(y, cur_predict)
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=None
            )
            tree.fit(X, gradients)
            tree_predictions = tree.predict(X)
            cur_predict -= self.learning_rate * tree_predictions
            self.trees.append(tree)
    
    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        else:
            X = np.array(X)
        predictions = np.full(X.shape[0], self.init_predict)
        for tree in self.trees:
            tree_predictions = tree.predict(X)
            predictions -= self.learning_rate * tree_predictions
        probs = self._sigmoid(predictions)
        return np.column_stack([1 - probs, probs])
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba[:, 1] > 0.5).astype(int)

In [22]:
gbdt = my_GBDTClassifier(
    number_of_trees =100,          
    learning_rate=0.1,
    max_depth=3,
    random_state=21
)
gbdt.fit(X_train, y_train)
gbdt_proba = gbdt.predict_proba(X_valid)[:,1]
auc = roc_auc_score(y_valid, gbdt_proba)
gini = 2 * auc - 1
gini

0.47180672590504025

## 7. Use LightGBM, Catboost, and XGBoost

### XGBoost (eXtreme Gradient Boosting)
### Суть: Реализация градиентного бустинга с акцентом на регуляризацию и эффективность.
Ключевые характеристики:
- Деревья строятся горизонтально. На каждом уровне рассматриваются все возможные разделения одновременно.
- Взвешенный квантильный эскиз для поиска разделений. Это алгоритм, который вместо перебора всех значений признака выбирает небольшое количество разделителей, аппроксимируя взвешенное распределение данных с гарантированной точностью.
### Основные параметры:
learning_rate, max_depth, lambda/alpha, subsample(доля строк), colsample_bytree(доля признаков для каждого дерева), n_estimators
### Особенности:
- Требует кодирования категорий
- Медленнее на больших данных
- Автоматическая обработка пропущенных значений
### DART (Dropouts meet Multiple Additive Regression Trees) — это специальный режим работы XGBoost, который применяет технику временного исключения из нейронных сетей к алгоритму градиентного бустинга.
Суть:
В обычном XGBoost каждое новое дерево учится исправлять ошибки всех предыдущих деревьев. Это может привести к переобучению на шумных данных и несбалансированному вкладу деревьев. На каждой итерации DART случайно исключает часть ранее построенных деревьев из ансамбля перед обучением нового дерева.

### LightGBM (Light Gradient Boosting Machine)
### Суть: Реализация градиентного бустинга с акцентом на скорость и эффективное использование памяти.
Ключевые характеристики:
- Дерево растет асимметрично, выбирая на каждом шаге лист с максимальным уменьшением потерь
- Сохраняет объекты с большими градиентами и случайную выборку объектов с малыми градиентами
— Объединяет взаимно исключающие признаки для уменьшения размерности
### Основные параметры: 
num_leaves, learning_rate, max_depth, min_data_in_leaf (минимальное количество данных в листе),feature_fraction(доля признаков для итерации), bagging_fraction(доля данных для  бутстрэп-агрегирования), lambda_l1/lambda_l2
### Особенности:
- Высокая скорость обучения
- Низкое потребление памяти
- Поддержка категориальных признаков без предварительного кодирования
- Риск переобучения на малых данных
### Linear Tree — это особенность LightGBM, которая вместо константных значений в листьях деревьев использует линейные модели (обычно линейную регрессию).
### Суть:
- В каждом листовом узле строится простая линейная модель на основе признаков, которые попали в этот лист
- Вместо простого усреднения целевых значений в листе используется линейная комбинация признаков
- Веса линейной модели обучаются градиентным спуском

Преимущества:
- Лучше работает с данными, имеющими линейные зависимости
- Может уменьшить количество необходимых деревьев
- Более точные предсказания на гладких функциях

Недостатки:
- Увеличивает риск переобучения
- Требует больше вычислительных ресурсов


### CatBoost (Categorical Boosting)
### Суть: Реализация градиентного бустинга с продвинутой обработкой категориальных признаков и защитой от переобучения.
Ключевые характеристики:
- Ordered Boosting — модификация алгоритма бустинга, устраняющая смещение предсказаний
- Симметричные деревья и одинаковые разделения на каждом уровне для эффективности вычислений (все узлы на одном уровне дерева используют одинаковые признаки и пороги для разделения)
- Автоматическая обработка категориальных признаков — встроенные механизмы работы с категориальными признаками
Основные параметры:
iterations, learning_rate, depth, l2_leaf_reg, random_strength, cat_features
### Особенности:
- Устойчивость к переобучению
- Низкая скорость обучения по сравнению с LightGBM
- Полное исключение утечек данных
- Автоматизация предобработки данных
### Ordered Target Encoding — это специальный алгоритм в CatBoost, который предотвращает переобучение при работе с категориальными признаками.
### Суть:
В стандартном Target Encoding статистика по категориям вычисляется по всей выборке, что приводит к утечкам. CatBoost решает эту проблему через Ordered Target Encoding, основанный на принципе временных рядов. Создается случайный порядок объектов, а затем для каждого объекта i вычисляется encoding только на основе объектов 1...(i-1). Процесс повторяется для нескольких случайных порядков


In [25]:
import optuna
from lightgbm import LGBMClassifier

/Users/daughtor/ml/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'grow_policy': trial.suggest_categorical('grow_policy', 
                        ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'rsm': trial.suggest_float('rsm', 0.5, 1.0),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 10),
        'random_seed':21,
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'verbose':False
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train,y_train)
    y_proba=model.predict_proba(X_valid)[:,1]
    auc = roc_auc_score(y_valid, y_proba)
    gini = 2 * auc - 1
    return gini
   

In [27]:
study1 = optuna.create_study(direction='maximize')
study1.optimize(objective, n_trials=50)

[I 2025-11-10 20:51:52,140] A new study created in memory with name: no-name-5939db32-31ba-41c0-9f76-7c7ce647d0a5
[I 2025-11-10 20:51:55,390] Trial 0 finished with value: 0.4165792983717753 and parameters: {'iterations': 812, 'learning_rate': 0.09232455172491126, 'l2_leaf_reg': 6.08039649440209, 'random_strength': 2.1850996214163723, 'border_count': 250, 'bagging_temperature': 0.22637070930510406, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 46, 'rsm': 0.5680671208463406, 'leaf_estimation_iterations': 9}. Best is trial 0 with value: 0.4165792983717753.
[I 2025-11-10 20:51:56,479] Trial 1 finished with value: 0.4855348215313451 and parameters: {'iterations': 503, 'learning_rate': 0.013745290980599864, 'l2_leaf_reg': 8.913931147114225, 'random_strength': 5.615408640722761, 'border_count': 206, 'bagging_temperature': 0.5797879218630412, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 31, 'rsm': 0.9687304460718158, 'leaf_estimation_iterations': 7}. Best is trial 1 with value: 0.4855

In [28]:
study1.best_trial.value

0.49861771194154003

In [29]:
best_params_study1 = study1.best_params
best_params_study1.update({'random_seed':21,'loss_function': 'Logloss','eval_metric': 'AUC','verbose':False})

In [30]:
model=CatBoostClassifier(**best_params_study1)
model.fit(X_train,y_train)
y_proba_train = model.predict_proba(X_train)[:, 1]
y_proba_valid = model.predict_proba(X_valid)[:, 1]
y_proba_test=model.predict_proba(X_test)[:,1]
gini_train = 2*roc_auc_score(y_train, y_proba_train)-1
gini_valid = 2*roc_auc_score(y_valid, y_proba_valid)-1
gini_test = 2*roc_auc_score(y_test, y_proba_test)-1

In [31]:
gini_train

0.6534327464925409

In [32]:
gini_valid

0.49861771194154003

In [33]:
gini_test

0.47458521834430045

In [34]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 512),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'max_bin': trial.suggest_int('max_bin', 32, 255),
        'random_seed':21,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1
    }
    model = LGBMClassifier(**params)
    model.fit(X_train,y_train)
    y_proba=model.predict_proba(X_valid)[:,1]
    auc = roc_auc_score(y_valid, y_proba)
    gini = 2 * auc - 1
    return gini
    

In [35]:
study2 = optuna.create_study(direction='maximize')
study2.optimize(objective, n_trials=50)

[I 2025-11-10 20:53:54,473] A new study created in memory with name: no-name-54823145-5d22-40c9-ab1f-dad013a5525c
[I 2025-11-10 20:53:57,879] Trial 0 finished with value: 0.4889578341918863 and parameters: {'n_estimators': 366, 'learning_rate': 0.010052777170948171, 'max_depth': 11, 'lambda_l1': 0.36156754387952883, 'lambda_l2': 5.237152209499805e-06, 'num_leaves': 378, 'min_data_in_leaf': 96, 'min_gain_to_split': 0.5850512136922393, 'feature_fraction': 0.6731091473859638, 'bagging_fraction': 0.5655892380135068, 'bagging_freq': 2, 'max_bin': 196}. Best is trial 0 with value: 0.4889578341918863.
[I 2025-11-10 20:53:59,711] Trial 1 finished with value: 0.48419796795289427 and parameters: {'n_estimators': 216, 'learning_rate': 0.01743443112536441, 'max_depth': 8, 'lambda_l1': 3.425535378542812e-08, 'lambda_l2': 2.307162162608978e-08, 'num_leaves': 501, 'min_data_in_leaf': 35, 'min_gain_to_split': 0.029232188229225464, 'feature_fraction': 0.7562976086920417, 'bagging_fraction': 0.637042165

In [36]:
study2.best_trial.value

0.49147146008589093

In [37]:
best_params_study2 = study2.best_params
best_params_study2.update({'random_seed':21,'objective': 'binary','metric': 'auc','verbose': -1})

In [38]:
model=LGBMClassifier(**best_params_study2)
model.fit(X_train,y_train)
y_proba_train = model.predict_proba(X_train)[:, 1]
y_proba_valid = model.predict_proba(X_valid)[:, 1]
y_proba_test=model.predict_proba(X_test)[:,1]
gini_train = 2*roc_auc_score(y_train, y_proba_train)-1
gini_valid = 2*roc_auc_score(y_valid, y_proba_valid)-1
gini_test = 2*roc_auc_score(y_test, y_proba_test)-1

In [39]:
gini_train

0.6250996041098382

In [40]:
gini_valid

0.49147146008589093

In [41]:
gini_test

0.5003762796842128

In [ ]:

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10), 
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10),  
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'max_bin': trial.suggest_int('max_bin', 32, 255),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 20), 
        'random_seed':21,
        'gamma': trial.suggest_float('gamma', 0, 5), 
        'max_delta_step': trial.suggest_float('max_delta_step', 0, 10),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'objective': 'binary:logistic',
        'eval_metric': 'auc',    
    }
    model = XGBClassifier(**params)
    model.fit(X_train,y_train)
    y_proba=model.predict_proba(X_valid)[:,1]
    auc = roc_auc_score(y_valid, y_proba)
    gini = 2 * auc - 1
    return gini


In [43]:
study3 = optuna.create_study(direction='maximize')
study3.optimize(objective, n_trials=50)

[I 2025-11-10 20:55:44,559] A new study created in memory with name: no-name-478e5687-10ed-422b-9409-a4e3b44c947e
/Users/daughtor/ml/.venv/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [20:55:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "random_seed" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-11-10 20:55:49,941] Trial 0 finished with value: 0.4907810132230208 and parameters: {'n_estimators': 537, 'learning_rate': 0.01150853747121557, 'max_depth': 9, 'reg_lambda': 4.224130456315874, 'reg_alpha': 9.621555870699327, 'subsample': 0.7536133676851808, 'colsample_bytree': 0.6628604212891396, 'max_bin': 252, 'grow_policy': 'lossguide', 'min_child_weight': 3.808222665199013, 'gamma': 0.18184635766444746, 'max_delta_step': 3.914239846235935, 'booster': 'gbtree'}. Best is trial 0 with value: 0.4907810132230208.
/Users/daughtor/ml/.venv/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [20:55:49] 

In [44]:
study3.best_trial.value

0.4944831742607365

In [45]:
best_params_study3 = study3.best_params
best_params_study3.update({'random_seed':21,'objective': 'binary:logistic','eval_metric': 'auc','verbose': -1})

In [46]:
model=XGBClassifier(**best_params_study3)
model.fit(X_train,y_train)
y_proba_train = model.predict_proba(X_train)[:, 1]
y_proba_valid = model.predict_proba(X_valid)[:, 1]
y_proba_test=model.predict_proba(X_test)[:,1]
gini_train = 2*roc_auc_score(y_train, y_proba_train)-1
gini_valid = 2*roc_auc_score(y_valid, y_proba_valid)-1
gini_test = 2*roc_auc_score(y_test, y_proba_test)-1

/Users/daughtor/ml/.venv/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [21:25:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "random_seed", "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [47]:
gini_train

0.6517914868697137

In [48]:
gini_valid

0.4944831742607365

In [49]:
gini_test

0.4731420178690664

### CatBoost показал минимальное преимущество благодаря встроенной обработке категориальных признаков, однако XGBoost продемонстрировал сравнимую точность за счет надежного level-wise роста деревьев, обеспечивающего стабильность. LightGBM также показал конкурентные результаты, компенсируя потенциальную склонность к переобучению исключительной скоростью работы и эффективным использованием памяти.


## 9. Implement the ExtraTreesClassifier

### ExtraTreesClassifier (Extremely Randomized Trees) — это ансамблевый метод, который использует максимально рандомизированные деревья решений для повышения разнообразия и уменьшения переобучения.
### Суть метода:
- Каждое дерево строится на всей выборке
- Разбиения выбираются случайно: случайный признак, случайный порог
- Из всех случайных разбиений выбирается лучшее
### Алгоритм:
- Для каждого узла случайно выбирается подмножество признаков
- Для каждого выбранного признака генерируется случайный порог разбиения
- Из всех случайных кандидатов выбирается лучшее разбиение
### Плюсы:
- Высокая скорость обучения (нет поиска оптимальных разбиений)
- Меньшая вероятность переобучения
- Меньшая чувствительность к шуму
- Эффективен на данных с большим количеством признаков
### Минусы:
- Может проигрывать в точности на малых выборках
- Сильная зависимость от числа деревьев

In [50]:
class my_ExtraTreesClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, 
                 random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.estimators_ = []
        self.n_classes_ = None
        self.random_state=random_state
        self.rng = np.random.RandomState(self.random_state)
        
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.n_classes_ = len(np.unique(y))
        self.estimators_ = []
        for i in range(self.n_estimators):
            tree = my_DecisionTreeClassifierRand(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                random_state=self.rng.randint(0, 1000)
            )
            n_samples = X.shape[0]
            indices = self.rng.choice(n_samples, size=n_samples, replace=True)
            X_bootstrap = X[indices]
            y_bootstrap = y[indices]
            tree.fit(X_bootstrap, y_bootstrap)
            self.estimators_.append(tree)   
        return self
    
    def predict_proba(self, X):
        X = np.array(X)
        all_proba = []
        for tree in self.estimators_:
            tree_proba = tree.predict_proba(X)
            all_proba.append(tree_proba)
        proba = np.mean(all_proba, axis=0)
        return proba
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)


In [51]:
model = my_ExtraTreesClassifier(n_estimators=10, max_depth=7,random_state=21,min_samples_split=2)
model.fit(X_train,y_train)
y_proba=model.predict_proba(X_valid)[:,1]
auc = roc_auc_score(y_valid, y_proba)
gini = 2 * auc - 1
gini
    

0.4637754732721251